# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from citipy import citipy

output_data_file = "../output_data/cities.csv"


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
airport_df = pd.read_csv("../output_data/cities.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
locations = airport_df[["Latitude", "Longitude"]]
humidity = airport_df["Humidity (%)"].astype(float)
humidity_percent = humidity/100

In [13]:
figure_layout = {
    'width': '1000px',
    'eight': '600px',
    'order': '1px solid black',
    'adding': '1px'}
fig = gmaps.figure(center=[25,10], zoom_level=2,layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px', order='1px solid black', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
hotel_df = airport_df[airport_df['Temperature (F)'] < 80]
hotel_df = airport_df[airport_df['Temperature (F)'] > 70]
hotel_df = airport_df[airport_df['Wind Speed (mph)'] < 10]
hotel_df = airport_df[airport_df['Cloudiness (%)'] < 0.001]

hotel_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
1,Alice Springs,AU,-23.70,133.88,41.00,44,0,2.24
6,Port Elizabeth,ZA,-33.92,25.57,60.80,82,0,9.17
15,Yulara,AU,-25.24,130.99,44.60,39,0,6.93
17,Henties Bay,NaN,-22.12,14.28,74.48,34,0,11.59
23,Hithadhoo,MV,-0.60,73.08,84.83,75,0,9.53
...,...,...,...,...,...,...,...,...
496,Vytegra,RU,61.01,36.45,70.97,54,0,3.74
501,Mitha Tiwana,PK,32.25,72.11,101.21,8,0,5.70
507,Port Macquarie,AU,-31.43,152.92,48.74,100,0,4.70
508,Aswān,EG,24.09,32.91,105.80,7,0,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Add a "Hotel Name" column to the DataFrame.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_df['Hotel Name'] = ""

# target_coordinates = coordinates
# target_search = "City"
# target_radius = 5000
# target_type = "lodging"

for index, row in hotel_df.iterrows():
    coordinates = str(hotel_df.loc[index,'Latitude']) +',' + str(hotel_df.loc[index,'Longitude'])
    params = {
        "location": coordinates,
        "radius": 5000,
        "type": "lodging",
        "key": g_key
        }
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name'];
    except (IndexError):
        continue
        
hotel_df.head()

# set up a parameters dictionary
# run a request using our params dictionary

C:\Users\Mitch\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
1,Alice Springs,AU,-23.70,133.88,41.00,44,0,2.24,Desert Palms Alice Springs
6,Port Elizabeth,ZA,-33.92,25.57,60.80,82,0,9.17,39 On Nile Guest House
15,Yulara,AU,-25.24,130.99,44.60,39,0,6.93,Desert Gardens Hotel - Ayers Rock Resort
17,Henties Bay,NaN,-22.12,14.28,74.48,34,0,11.59,Fishermans Guest House
23,Hithadhoo,MV,-0.60,73.08,84.83,75,0,9.53,Scoop Guest House


In [27]:
# NOTE: Do not change any of the code in this cell
narrowed_city_df = hotel_df
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Contry}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

KeyError: 'Contry'

In [28]:
# Add marker layer ontop of heat map
df= pd.read_csv(output_data_file)
locations = df[["Latitude", "Longitude"]]
Humidity = df["Humidity (%)"]

figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=[25,10], zoom_level=2,layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity)
fig.add_layer(heat_layer)

markers = gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)
fig.add_layer(markers)

fig

# Display Map

NameError: name 'hotel_locations' is not defined